In [44]:
# import the Biopython module
from Bio import SeqIO

# import default dict for the kmer collections
from collections import defaultdict

# example
seq = "TTGAAAGAAAAACAATTTTG"
k_dict = defaultdict(int)
for idx in range(0, len(seq)-13):
    kmer = seq[idx:(idx+14)]
    if kmer in k_dict:
        k_dict[kmer] = k_dict.get(kmer,0)+1
    else:
        k_dict[kmer] = 1
print(len(k_dict))

def count_kmers_from_fasta(fasta_file, k): 
    #read the input file
    fasta_entry = SeqIO.parse(open(fasta_file), 'fasta')
    # make empty dict
    kmers_dict = defaultdict(int)
    # read each contig separately
#     length = 0
#     n = 0
    for contig in fasta_entry:
        sequence = str(contig.seq)
#         length  += len(sequence)
#         n +=1
        # obtain kmers across the contig seq 
        for idx in range(0, len(sequence)-(k-1)): # last kmer is the last k nt in the sequence
            kmer = sequence[idx:(idx+k)]
            # store kmers to a dict if doesn't already exist, but increase count if it already exists
            if kmer in kmers_dict:
                kmers_dict[kmer] = kmers_dict.get(kmer,0)+1
            else:
                kmers_dict[kmer] = 1
    # count the occurences of kmers the file
    return len(kmers_dict)
# , length, n
    
# read in the DNA sequence from the above FASTA files
# count the occurences of 14-mers in each of the four files
files = ("s_pneumoniae_genomes/14412_3#82.contigs_velvet.fa", 
         "s_pneumoniae_genomes/14412_3#84.contigs_velvet.fa",
         "s_pneumoniae_genomes/R6.fa","s_pneumoniae_genomes/TIGR4.fa" )
for file in files:
    print(count_kmers_from_fasta(file, 14))



7
1780322
1770662
1952351
2027930


In [43]:
1832845-(14*13)

1832663

In [71]:
# calculates the Jaccard distance between two of the input samples
# example
A = ["TTGAAAGAAAAACA", "TGAAAGAAAAACAA",
     "GAAAGAAAAACAAT", "AAAGAAAAACAATT",
     "AAGAAAAACAATTT", "AGAAAAACAATTTT", 
     "GAAAAACAATTTTG"]
B = ["TTGAAAGAAAAACA", "TGAAAGAAAAACAA", 
     "GAAAGAAAAACAAT", "AAAGAAAAACAATT",
     "GAAAAACAATTTTG", "CTCGATCCATGTAT", 
     "TCGATCCATGTATG"]

dictA = defaultdict(int)
dictB = defaultdict(int)
for kmer in A:
    if kmer in dictA:
        dictA[kmer] = dictA.get(kmer,0)+1
    else:
        dictA[kmer] = 1
for kmer in B:
    if kmer in dictB:
        dictB[kmer] = dictB.get(kmer,0)+1
    else:
        dictB[kmer] = 1
# count shared and unique kmers between samples
AnB, AuB = 0, 0
for k in dictA :
    if k in dictB:
        AnB +=1
        AuB +=1
        del dictB[k] # remove the shared kmer from the dict
    else:
        AuB +=1
# add the unique kmers in dictB to the union
AuB += len(dictB)
print(f'AnB: {AnB}, AuB: {AuB}')

# calculate the jaccard index
j_index = AnB/AuB
print(f'jaccard index : {j_index}')

# calculate the jaccard distance
j_dist = 1-j_index
print(f'jaccard distance: {j_dist}') 

AnB: 5, AuB: 9
jaccard index : 0.5555555555555556
jaccard distance: 0.4444444444444444


In [50]:
# calculates the Jaccard distance between two of the input samples

def fasta_to_kmers(fasta, k_value):
    fasta_entry = SeqIO.parse(open(fasta), 'fasta')
    kmers_dict = defaultdict(int)
    # treat each contig as separate
    for contig in fasta_entry:
        sequence = str(contig.seq)
        # obtain kmers across the contig seq 
        for idx in range(0, len(sequence)-(k_value-1)): # last kmer is the last k nt in the sequence
            kmer = sequence[idx:(idx+k_value)]
            # store kmers to a dict if doesn't already exist, but increase count if it already exists
            if kmer in kmers_dict:
                kmers_dict[kmer] = kmers_dict.get(kmer,0)+1
            else:
                kmers_dict[kmer] = 1
    return kmers_dict

def cal_jaccard_distance(fasta1, fasta2, k): 
    #read the input files and obtain kmers
    kmers_dictA = fasta_to_kmers(fasta1, k)
    kmers_dictB = fasta_to_kmers(fasta2, k)
    
    # count shared and unique kmers between samples
    AnB, AuB = 0, 0
    for kmer in kmers_dictA :
        if kmer in kmers_dictB: # shared kmers between the two samples
            AnB +=1
            AuB +=1
            del kmers_dictB[kmer] # remove the shared kmer from the B dict
        else: # unique kmers from A
            AuB +=1
    # add the unique kmers in B to the union
    AuB += len(kmers_dictB)
#     print(f'AnB: {AnB}, AuB: {AuB}')

    # calculate the jaccard index
    j_index = AnB/AuB
    print(f'jaccard index : {j_index}')

    # calculate the jaccard distance
    j_dist = 1-j_index
#     print(f'jaccard distance: {j_dist}') 
    return j_dist
    
cal_jaccard_distance("s_pneumoniae_genomes/14412_3#82.contigs_velvet.fa", 
         "s_pneumoniae_genomes/14412_3#84.contigs_velvet.fa", 14)

jaccard index : 0.43923666207454476


0.5607633379254553

In [68]:
import xxhash
# some 14-mers for this example
kmers = ["TTGAAAGAAAAACA","TGAAAGAAAAACAA","GAAAGAAAAACAAT",
         "AAAGAAAAACAATT", "AAGAAAAACAATTT", "AGAAAAACAATTTT",
         "GAAAAACAATTTTG"]
# calculate a hash of the first k-mer, this comes back as a hex string.
print(xxhash.xxh32(kmers[0]).hexdigest())

3ad3bb24


In [69]:
# Run on all the k-mers
hash1 = []
for kmer in kmers:
    hash1.append(xxhash.xxh32(kmer).hexdigest())
hash1

['3ad3bb24',
 'dc529a4b',
 '45fab9d4',
 '2d857675',
 '055d3d72',
 'c3967606',
 'e864aa15']

In [70]:
# Confirm the same hashes are generated on repeated runs (so they are comparable between samples)
hash2 =[]
for kmer in kmers:
    hash2.append(xxhash.xxh32(kmer).hexdigest())
hash2==hash1

True

In [81]:
# get dictionary
dict82 = fasta_to_kmers("s_pneumoniae_genomes/14412_3#82.contigs_velvet.fa", 14)
dict84 = fasta_to_kmers("s_pneumoniae_genomes/14412_3#84.contigs_velvet.fa", 14)

def dict_hash(the_dict):
    result = []
    for key in the_dict:
        result.append(xxhash.xxh32(key).hexdigest())
    return result
# loop
files = ("s_pneumoniae_genomes/14412_3#82.contigs_velvet.fa", "s_pneumoniae_genomes/14412_3#84.contigs_velvet.fa")
for file in files:
    dict_file = fasta_to_kmers(file, 14)
    hashes = dict_hash(dict_file)
    sorted_hashes = sorted(hashes)
    s = 1000
    smallest = sorted_hashes[0:1000]
    #save smallest to a text file
    textfile = open("a_file.txt", "w")
    for element in a_list:
        textfile.write(element + "\n")
    textfile.close()

['055d3d72',
 '2d857675',
 '3ad3bb24',
 '45fab9d4',
 'c3967606',
 'dc529a4b',
 'e864aa15']